In [1]:
import newspaper
from newspaper import news_pool
import googlesearch
import numpy as np
import pickle
from tqdm import tqdm
import timeout_decorator

In [2]:
# Doesn't work very well for chinese...
news_translations = ["news", "消息", "Новости"]
urls = []
for url in googlesearch.search("消息", lang="zh", num_results=10):
  urls.append(url)
print(urls)

['https://news.un.org/zh/news', 'https://baike.baidu.com/item/%E6%B6%88%E6%81%AF/1619218', 'https://cn.chinadaily.com.cn/gtx/5d47c3fba31099ab995d74b7/cankaoxiaoxi', 'https://en.wiktionary.org/wiki/%E6%B6%88%E6%81%AF', 'https://aws.amazon.com/cn/message-queue/', 'https://zh.wikipedia.org/zh-hans/%E5%8F%82%E8%80%83%E6%B6%88%E6%81%AF', 'https://support.google.com/chat/answer/7652736?hl=zh-Hans&co=GENIE.Platform%3DAndroid', 'https://slack.com/intl/zh-cn/help/articles/202288908-%E8%AE%BE%E7%BD%AE%E4%BD%A0%E7%9A%84%E6%B6%88%E6%81%AF%E6%A0%BC%E5%BC%8F', 'https://context.reverso.net/%E7%BF%BB%E8%AF%91/%E4%B8%AD%E6%96%87-%E8%8B%B1%E8%AF%AD/%E6%B6%88%E6%81%AF', 'https://slack.com/intl/zh-cn/help/articles/203457187-%E8%87%AA%E5%AE%9A%E4%B9%89%E6%B6%88%E6%81%AF%E5%92%8C%E6%96%87%E4%BB%B6%E7%95%99%E5%AD%98', 'https://slack.com/intl/zh-cn/help/articles/202288908-%E8%AE%BE%E7%BD%AE%E4%BD%A0%E7%9A%84%E6%B6%88%E6%81%AF%E6%A0%BC%E5%BC%8F', 'https://support.google.com/chat/answer/7652736?hl=zh-Hans&co=GE

In [5]:
# Handpicked list from https://github.com/yavuz/news-feed-list-of-countries

cn_urls = ["http://news.baidu.com/", "http://people.com.cn/", "https://www.chinanews.com.cn/",
           "http://www.cnr.cn/", "https://cn.chinadaily.com.cn"]

en_urls = ["https://www.cnn.com/", "https://www.nytimes.com/", "https://www.foxnews.com/",
           "https://www.usatoday.com/", "https://abcnews.go.com/"]

# lots of removed articles from https://novayagazeta.ru/ after journalists were murdered
# (Last major independent media org in russia to shut down...)
rs_urls = ["https://news.mail.ru/", "https://novayagazeta.ru/", "https://ria.ru/",
           "https://www.ntv.ru/", "http://rbc.ru/"]

In [6]:
# Maybe switch to newspaper3k's automatic language detection since some articles may
# not be in the expected language
lang_urls = [cn_urls, en_urls, rs_urls]
lang_abvs = ["zh", "en", "ru"]
built_papers = []
articles = []

tqdm.write("Building papers...")
for i in tqdm(range(len(lang_urls))):
  urls = lang_urls[i]
  abv = lang_abvs[i]
  for url in urls:
    paper = newspaper.build(url, memoize_articles=False)
    built_papers.append(paper)

Building papers...


100%|██████████| 3/3 [03:40<00:00, 73.39s/it] 


In [8]:
tqdm.write("Downloading...")
news_pool.set(built_papers, threads_per_source=2) # (15*2) = 30 threads total
news_pool.join()
tqdm.write("Done!")

Downloading...


In [3]:
@timeout_decorator.timeout(5, timeout_exception=StopIteration)
def saveArticle(article):
  article.parse()
  articles.append({"abv": abv, "url": article.url, "title": article.title, 
                     "authors": article.authors, "date": article.publish_date,
                    "text": article.text})
  np.save("articles2.npy", articles)

In [ ]:
# Takes around a day to finish running...
for paper in built_papers:
  for article in tqdm(paper.articles):
    try:
      saveArticle(article)
    except StopIteration:
      print("Timed out while parsing article")
      continue

In [ ]:
articles = np.load("articles.npy2", allow_pickle=True)

# Size of articles changes slightly between runs, even though memoize_articles=False
print(len(articles))
print(articles[2000])
